In [1]:
import torch
import torch.nn as nn

In [2]:
class CBOW(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, literal_to_ix):
        super(CBOW, self).__init__()
        # out: 1 x embedding_dim
        self.vocab_size = vocab_size
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.literal_to_ix = literal_to_ix
        self.linear1 = nn.Linear(embedding_dim, 128)
        self.activation_function1 = nn.ReLU()
        
        # out: 1 x vocab_size
        self.linear2 = nn.Linear(128, vocab_size)
        self.activation_function2 = nn.LogSoftmax(dim=-1)
        
    def forward(self, inputs):
        embeds = sum(self.embeddings(inputs)).view(1, -1)
        out = self.linear1(embeds)
        out = self.activation_function1(out)
        out = self.linear2(out)
        out = self.activation_function2(out)
        return out
    
    def get_literal_embedding(self, literal):
        ix = torch.tensor([self.literal_to_ix[literal]])
        return self.embeddings(ix)

    def get_embeddings(self):
        ix = torch.tensor([i for i in range(self.vocab_size)])
        return self.embeddings(ix)

In [3]:
# utils 
def make_context_vector(context, literal_to_idx):
    if 0 in context:
        print(context)
    idxs = [literal_to_idx[l] for l in context]
    return torch.tensor(idxs, dtype=torch.long)


def read_sat(sat_path):
    with open(sat_path) as f:
        sat_lines = f.readlines()
        header = sat_lines[0]
        header_info = header.replace("\n", "").split(" ")
        num_vars = int(header_info[-2])
        num_clauses = int(header_info[-1])

        sat = [[int(x) for x in line.replace(' 0\n', '').split(' ')]
               for line in sat_lines[1:]]

        return sat, num_vars, num_clauses

In [4]:
# data preprocessing

# name = 'aes_64_1_keyfind_1.processed.cnf'
name = 'aes_32_3_keyfind_2.processed.cnf'

sat_path = f'./dataset/formulas/{name}'
sat_instance, num_vars, num_clauses = read_sat(sat_path)
vocab_size = num_vars * 2

data = []
for clause in sat_instance:
    clause_len = len(clause)
    for i in range(clause_len):
        context = [clause[x] for x in range(clause_len) if x != i]
        target = clause[i]
        data.append((context, target))

print(f'data size: {len(data)}')

data size: 7772


In [5]:
# model setting

EMDEDDING_DIM = 50

literal_to_ix = {}
for i in range(1, num_vars + 1):
    literal_to_ix[i] = 2 * i - 2
    literal_to_ix[-i] = 2 * i - 1

model = CBOW(vocab_size, EMDEDDING_DIM, literal_to_ix)
loss_function = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

# training
for epoch in range(1):
    total_loss = 0
    for context, target in data:
        print(context, target)
        context_vector = make_context_vector(context, literal_to_ix)
        log_probs = model(context_vector)
        total_loss += loss_function(log_probs, torch.tensor([literal_to_ix[target]]))
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()
    
    if epoch % 10 == 0:
        print(epoch, total_loss.item())

[-2, -3, 4] -1
[-1, -3, 4] -2
[-1, -2, 4] -3
[-1, -2, -3] 4
[-2, -4, -5] -1
[-1, -4, -5] -2
[-1, -2, -5] -4
[-1, -2, -4] -5
[4, -5] -6
[-6, -5] 4
[-6, 4] -5
[2, 4, -5] -1
[-1, 4, -5] 2
[-1, 2, -5] 4
[-1, 2, 4] -5
[1, -2, -7] 6
[6, -2, -7] 1
[6, 1, -7] -2
[6, 1, -2] -7
[3, -4, -7] 2
[2, -4, -7] 3
[2, 3, -7] -4
[2, 3, -4] -7
[-3, -5, -7] 1
[1, -5, -7] -3
[1, -3, -7] -5
[1, -3, -5] -7
[2, -3, 5, -7] -1
[-1, -3, 5, -7] 2
[-1, 2, 5, -7] -3
[-1, 2, -3, -7] 5
[-1, 2, -3, 5] -7
[-4, 7] 6
[6, 7] -4
[6, -4] 7
[3, -4, 7] -2
[-2, -4, 7] 3
[-2, 3, 7] -4
[-2, 3, -4] 7
[-1, 2, -8] 6
[6, 2, -8] -1
[6, -1, -8] 2
[6, -1, 2] -8
[-3, -8] -2
[-2, -8] -3
[-2, -3] -8
[-4, -8] -1
[-1, -8] -4
[-1, -4] -8
[2, 4, -8] 1
[1, 4, -8] 2
[1, 2, -8] 4
[1, 2, 4] -8
[3, -5, -8] 2
[2, -5, -8] 3
[2, 3, -8] -5
[2, 3, -5] -8
[5, -8] -2
[-2, -8] 5
[-2, 5] -8
[-3, -4, 5, -8] -6
[-6, -4, 5, -8] -3
[-6, -3, 5, -8] -4
[-6, -3, -4, -8] 5
[-6, -3, -4, 5] -8
[3, 7, -8] -1
[-1, 7, -8] 3
[-1, 3, -8] 7
[-1, 3, 7] -8
[-2, 3, 8] 6
[6, 3,

In [6]:
# test the embedding
print(model.get_literal_embedding(91))
embeddings = model.get_embeddings()
torch.save(embeddings, f'./model/embeddings/{name}.pt')

tensor([[ 1.2355e+00,  1.5368e+00, -3.1604e-01, -6.5582e-01, -4.1272e-01,
          1.3567e+00, -5.6561e-01,  2.2593e-01, -8.5319e-01,  5.2873e-01,
         -1.4715e-01, -1.8518e+00, -2.5861e+00,  9.4754e-01,  1.8165e+00,
         -2.1879e-01,  1.2599e+00, -6.6656e-01,  3.3519e-01,  1.1566e+00,
          5.1483e-01,  1.2767e-01,  9.7376e-01,  9.8673e-01, -7.1446e-01,
          3.5617e-01, -6.9461e-01,  1.3161e+00,  1.5208e+00, -5.4191e-01,
          1.5162e-01, -6.8128e-01, -4.0280e-01, -6.5493e-01, -1.4239e+00,
         -8.9439e-01,  2.0646e-01, -1.2312e+00, -1.2107e+00, -4.0627e-01,
         -1.2328e+00,  8.6893e-04,  3.1391e-01, -8.8832e-01,  5.3872e-01,
          1.4615e+00,  8.2907e-01, -1.9508e-02,  4.0461e-02,  1.1197e-01]],
       grad_fn=<EmbeddingBackward0>)


In [7]:
embeddings[180]

tensor([ 1.2355e+00,  1.5368e+00, -3.1604e-01, -6.5582e-01, -4.1272e-01,
         1.3567e+00, -5.6561e-01,  2.2593e-01, -8.5319e-01,  5.2873e-01,
        -1.4715e-01, -1.8518e+00, -2.5861e+00,  9.4754e-01,  1.8165e+00,
        -2.1879e-01,  1.2599e+00, -6.6656e-01,  3.3519e-01,  1.1566e+00,
         5.1483e-01,  1.2767e-01,  9.7376e-01,  9.8673e-01, -7.1446e-01,
         3.5617e-01, -6.9461e-01,  1.3161e+00,  1.5208e+00, -5.4191e-01,
         1.5162e-01, -6.8128e-01, -4.0280e-01, -6.5493e-01, -1.4239e+00,
        -8.9439e-01,  2.0646e-01, -1.2312e+00, -1.2107e+00, -4.0627e-01,
        -1.2328e+00,  8.6893e-04,  3.1391e-01, -8.8832e-01,  5.3872e-01,
         1.4615e+00,  8.2907e-01, -1.9508e-02,  4.0461e-02,  1.1197e-01],
       grad_fn=<SelectBackward0>)